In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.linear_model import MultiTaskLasso
import json
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from preprocessing import *

In [ ]:
# Charger les fichiers
X_file = "data/X_matrix.csv"
Y_file = "data/Y_matrix.csv"

print("Chargement des données...")
X = pd.read_csv(X_file)
Y = pd.read_csv(Y_file)

In [ ]:
X = X.drop(columns=["Yeast_ID"]).fillna(0)
Y = Y.drop(columns=["Yeast_ID"]).fillna(Y.drop(columns=["Yeast_ID"]).mean())

X_pre, Y_pre = preprocessed_data(X, Y)

In [ ]:
# Division des données en ensembles d'entraînement et de test
print("Division des données en ensembles d'entraînement et de test...")
sss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
train_index, test_index = next(sss.split(X_pre, Y_pre))

X_train, X_test = X_pre.iloc[train_index, :], X_pre.iloc[test_index, :]
y_train, y_test = Y_pre.iloc[train_index, :], Y_pre.iloc[test_index, :]

# Affichage des dimensions des ensembles divisés
print(f"Dimensions de X_train : {X_train.shape}, X_test : {X_test.shape}")
print(f"Dimensions de y_train : {y_train.shape}, y_test : {y_test.shape}")

In [ ]:
# Définir les paramètres pour la recherche sur grille
print("Définition des paramètres pour GridSearch...")
lasso = MultiTaskLasso(random_state=42, max_iter=10000)
tuned_parameters = {'alpha': [0.1, 0.5, 0.75, 1.0, 2.0, 5.0]}  # Valeurs pour la régularisation

In [ ]:
# Lancer GridSearch avec validation croisée
print("Lancement de la recherche sur grille avec validation croisée...")
grid_search = GridSearchCV(lasso, tuned_parameters, cv=3, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)


In [ ]:
# Entraînement final avec le meilleur alpha
best_alpha = grid_search.best_params_['alpha']
print(f"Meilleur paramètre alpha: {best_alpha}")
final_model = MultiTaskLasso(alpha=best_alpha, random_state=42, max_iter=10000)
final_model.fit(X_train, y_train)

In [ ]:
# Prédictions
print("Prédictions sur les données de test...")
y_pred = final_model.predict(X_test)

In [ ]:
# Sauvegarder les résultats
pd.DataFrame(y_pred, index=Y.iloc[test_index, :].index, columns=Y.columns).to_csv('results/y_test_predicted_LASSO.csv')
pd.DataFrame(y_test, index=Y.iloc[test_index, :].index, columns=Y.columns).to_csv('results/y_test_true_LASSO.csv')

In [ ]:
# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"R² Score: {r2:.6f}")

In [ ]:
# Sauvegarder les coefficients
coefficients_df = pd.DataFrame(final_model.coef_, index=Y_pre.columns, columns=X_pre.columns)
coefficients_df.to_csv('results/LASSO_feature_importances.csv')

In [ ]:
# Visualisation des résultats
print("Affichage des résultats...")
top_features = coefficients_df.abs().mean(axis=0).sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 6))
plt.barh(top_features.index, top_features.values, color="skyblue")
plt.xlabel("Importance moyenne des coefficients absolus")
plt.ylabel("Mutation / Variation")
plt.title("Top 10 Mutations et CNVs Impactant le YPD Doubling Time")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Enregistrement des métadonnées
with open(f'results/LASSO_model_metadata.json', 'w+') as f:
    metadata = {
        'best_alpha': best_alpha,
        'mse': mse,
        'r2_score': r2,
        'training_samples': len(X_train),
        'testing_samples': len(X_test)
    }
    json.dump(metadata, f)
print("Les métadonnées du modèle ont été sauvegardées.")